## MT571 - Projeto de Implementação Computacional

In [1]:
#importacao de bibliotecas
using LinearAlgebra
using ControlSystems
using Plots
using Random
using Statistics

include("ARX_ON.jl") #pacote próprio

ARX_K (generic function with 1 method)

In [2]:
#parâmetros arbitrários de um sistema
m = 1
c = 2
k = 50

A = [
    0 1;
    -k/m -c/m
]

B = [0 ; 1/m]

C = [1 0]

D = 0
#modelo de espaço de estados
sys = ss(A,B,C,D)

#condicoes numericas
dt = 2e-5;
maxT = 6;
ns = round(Int,maxT/dt)+1;
t = 0:dt:maxT;

#estado inicial
x0 = [0; 0];

#força e geração do sinal de saída
U = ones(1,ns);#PulseGen(ns); 
Y,_,_ = lsim(sys,U,t,x0);

In [3]:
#parâmetros do ARX e dos otimizadores
na= 2
nb = 1
η = (1:1000)*1e-4
tol = 1e-5;

In [8]:
sysARX,or_Coef = ARX_K(Y,U,na,nb,dt); #geração do sistema e dos coeficientes
Y_or,_,_ = lsim(sysARX,U,t,x0) #resposta do ARX original
θ = [or_Coef[2],or_Coef[3],or_Coef[1]] #implementação dos meus coeficientes, na IC, foi feita em uma ordem inversa a deste trabalho. 

png(
    plot(

    t, 
    [Y' Y_or'],
    labels=["Sinal Original" "ARX"],
    ylabel = "Posição (m)", 
    xlabel="Tempo (s)"
),
"original_total"
)

"original_total.png"

In [9]:
Arduino = ControllerInit(0.01,na,nb,tol); #inicialização da estrutura do microcontrolador
grad_Bulid(Arduino,ns,Y,U) #geração das matrizes auxiliares do gradiente da MSE

In [19]:
norma = zeros(1000)
for i = 1:1000
    Arduino.θ = [0,0,0]#rand(na+nb);
    Arduino.η = η[i];
    GDS(Arduino);
    norma[i] = norm(θ-Arduino.θ,1)/norm(θ,1);
end

In [20]:
grafico = plot(
    (1:1000)*1e-4, 
    norma, 
    labels="Erro na Norma",
    ylabel = "ϵ", 
    xlabel = "η",
    title  = "Erro de norma ϵ x Learning Rate η - GDS",
);

png(grafico,"000-GDS")

"000-GDS.png"

In [21]:
norma2 = zeros(1000)
for i = 1:1000
    Arduino.θ = [0,0,0]#rand(na+nb);
    Arduino.η = η[i];
    AdamRun(Arduino);
    norma2[i] = norm(θ-Arduino.θ,1)/norm(θ,1);
end

In [22]:
grafico = plot(
    (1:1000)*1e-4, 
    norma2,  
    labels="Erro na Norma",
    ylabel = "ϵ", 
    xlabel = "η",
    title  = "Erro de norma ϵ x Learning Rate η - ADAM",
);

png(grafico,"000-adam")


"000-adam.png"